In [ ]:
import functools
import pandas as pd
import pickle
from analyzer.analysis_util import get_step_time
from analyzer.metatypes import OpType
from analyzer.simulator import simulate
from analyzer.to_timeline import dump_timeline

# Make sure you have completed the step 2 in README or 
# run `python3 ./analyzer/wia.py --metadata ./data/meta-ST.yaml --trace ./data/trace-ST.parquet --dump-dir ./logs/case-study-analysis --root ./logs/case-study-analysis` 
# before running this script.


df_ori = pd.read_parquet('./logs/case-study-analysis/dataframe/ST-0-no-delay.parquet')
step_time_ori = get_step_time(df_ori)
with open('./logs/case-study-analysis/dependency_model/ST-0.pkl', 'rb') as f:
    dependency_model = pickle.load(f)
dep_on, comm_groups, op2gid, need_dp_sync = dependency_model

sim_func = functools.partial(simulate,
                                dep_on,
                                comm_groups,
                                op2gid,
                                need_dp_sync,
                                step_starts=df_ori.groupby('step')['start_ts'].min(),
                                inplace=True)
print("Dump original timeline. Can visualize with https://ui.perfetto.dev/")
dump_timeline(f'./logs/timeline-ori.json.gz',
                df_ori,
                0,
                999999,
                reset_step_start=True)

dump original timeline. Can visualize with https://ui.perfetto.dev/
Dumping to ./logs/timeline-ori.json.gz


In [27]:
# what-if experiment 1: setting all PP-comm time to 0

df_new = df_ori.copy()
df_new.loc[df_new.optype.isin(OpType.p2p_optypes()), 'transfer_duration'] = 0
df_new, step_time_new = sim_func(df_new)
print('What-if experiment 1: setting all PP-comm time to 0 leads to {}X speedup'.format(
    step_time_ori.step_time.mean() / step_time_new.step_time.mean()
))
dump_timeline(f'./logs/timeline-what-if-exp1.json.gz',
                df_new,
                0,
                999999,
                reset_step_start=True)


What-if experiment 1: setting all PP-comm time to 0 leads to 1.0012516774513014X speedup
Dumping to ./logs/timeline-what-if-exp1.json.gz


In [28]:
# what-if experiment 2: setting all DP-comm time to 0

df_new = df_ori.copy()
df_new.loc[df_new.optype.isin(OpType.dp_comm_optypes()), 'transfer_duration'] = 0
df_new, step_time_new = sim_func(df_new)
print('What-if experiment 2: setting all DP-comm time to 0 leads to {}X speedup'.format(
    step_time_ori.step_time.mean() / step_time_new.step_time.mean()
))
dump_timeline(f'./logs/timeline-what-if-exp2.json.gz',
                df_new,
                0,
                999999,
                reset_step_start=True)


What-if experiment 2: setting all DP-comm time to 0 leads to 1.0392968710937196X speedup
Dumping to ./logs/timeline-what-if-exp2.json.gz
